In [1]:
#-*- coding:utf-8 -*-

import requests, os
import bs4
from bs4 import BeautifulSoup
from urllib import parse
import re
import lxml
import json
import sys

In [2]:
# return the list of shortcode from html code
def get_shortcode_list(soup):
    soup_script = soup.find_all("script")[3].string
    soup_script = re.split(':|,|\"|\'|\{|\}|\[|\]',soup_script)
    shortcodes = []
    temp = 0
    while 1:
        if 'shortcode' in soup_script[int(temp):]:

            new_shortcode = soup_script.index('shortcode', temp) + 3
            shortcodes.append(soup_script[new_shortcode])
            temp = new_shortcode + 1
        else :
            print("Get ", len(shortcodes), " shortcodes")
            return shortcodes

        
# return the text from html code
def get_text(page):
    script = page.find_all("script")[3].string
    script_split = re.split(':|,|\"|\'|\{|\}|\[|\]',script)  
    text_index = script_split.index('text') + 3
    text = script_split[text_index]  
    
    text = text.replace('\\n','')
    
    text = text.encode('utf-8', 'surrogatepass')
    text = text.decode('unicode_escape')
    
    #print(text)
    
    return text  

# return the user_id and number of likes from html code
def get_id_likes(description):
    data = re.split('on Instagram:', description)
    
    data = re.split(",|\(|\)|@|Likes|Comments|-| ",data[0])
    while '' in data : data.remove('')
    
    likes = data[0]
    user_id = data[-1]
    
    return user_id, likes

# return the text, user_id, number of likes from html code
def get_labels(page):
    for tag in page.select('meta[property*=image]'):
        image_url = tag['content']
            
    for tag in page.select('meta[property*=og:description]'):
        description = tag['content']

    text = get_text(page)
    user_id, likes = get_id_likes(description)
    
    return text, user_id, likes

# save image in page
def save_image(page):
    for tag in page.select('meta[property*=image]'):
        image_url = tag['content']
                
    pic_response = requests.get(image_url).content
    img_name = "./dataset/img/" + word + "_" + shortcode + ".jpg"
    
    with open(img_name, 'wb') as f:
        f.write(pic_response)
    print("file saved: ", img_name)    

In [3]:
word = input("\n** 원하는 태그를 입력하세요: \n>>> ")

# get the url from word(tag)
tag_url = "www.instagram.com/explore/tags/" + word
tag_url = "http://" + parse.quote(tag_url)
print(tag_url)

# get html code
response = requests.get(tag_url).text
soup = BeautifulSoup(response, 'lxml')

# get the list of shortcode
shortcodes = get_shortcode_list(soup)


** 원하는 태그를 입력하세요: 
>>> 서울N타워
http://www.instagram.com/explore/tags/%EC%84%9C%EC%9A%B8N%ED%83%80%EC%9B%8C
Get  31  shortcodes


In [4]:
i = 1
file_name = "./dataset/label.json"

for shortcode in shortcodes:
    print(i, "th ")
    i = i + 1  
    label_data = []
    
    try:
        # get html code
        page_url = "https://www.instagram.com/p/" + shortcode + "/"
        print(page_url)
        response = requests.get(page_url).content
        page = BeautifulSoup(response, 'lxml')        
        
        # get label data
        text, user_id, likes = get_labels(page)
        
        # save label data 
        output = {"shortcode":shortcode, "label":word, "likes":likes, "user_id":user_id, "text":text}
        label_data.append(output)
        
        if not os.path.isfile(file_name):
            with open(file_name, mode='w', encoding='UTF-16', errors='ignore') as f:
                new_feed = json.dumps(label_data, ensure_ascii=False, indent=2)
                f.write(new_feed)
        else:
            with open(file_name, encoding='UTF-16') as feedsjson:
                old_feeds = json.load(feedsjson)
            old_feeds.append(label_data)
            with open(file_name, mode='w', encoding='UTF-16', errors='ignore') as f:
                new_feed = json.dumps(old_feeds, ensure_ascii=False, indent=2)
                f.write(new_feed) 

        # save image files
        save_image(page)
            
        print("Okay\n")
        
    except:
        print("Unexpected error:", sys.exc_info()[0], sys.exc_info()[1])
        print("Not okay\n")

1 th 
https://www.instagram.com/p/Bp6OVrun1Wj/
file saved:  ./dataset/img/서울N타워_Bp6OVrun1Wj.jpg
Okay

2 th 
https://www.instagram.com/p/Bp5_dl1AJOp/
file saved:  ./dataset/img/서울N타워_Bp5_dl1AJOp.jpg
Okay

3 th 
https://www.instagram.com/p/Bp54I0CAxCh/
file saved:  ./dataset/img/서울N타워_Bp54I0CAxCh.jpg
Okay

4 th 
https://www.instagram.com/p/Bp4iAwrHtCJ/
file saved:  ./dataset/img/서울N타워_Bp4iAwrHtCJ.jpg
Okay

5 th 
https://www.instagram.com/p/Bp4cmOBjYeh/
file saved:  ./dataset/img/서울N타워_Bp4cmOBjYeh.jpg
Okay

6 th 
https://www.instagram.com/p/Bp4B_hQAH7P/
file saved:  ./dataset/img/서울N타워_Bp4B_hQAH7P.jpg
Okay

7 th 
https://www.instagram.com/p/Bp3-_guAf60/
file saved:  ./dataset/img/서울N타워_Bp3-_guAf60.jpg
Okay

8 th 
https://www.instagram.com/p/Bp3e3SogS9s/
file saved:  ./dataset/img/서울N타워_Bp3e3SogS9s.jpg
Okay

9 th 
https://www.instagram.com/p/Bp3LjOEASzv/
file saved:  ./dataset/img/서울N타워_Bp3LjOEASzv.jpg
Okay

10 th 
https://www.instagram.com/p/Bp2-7B1A2Lg/
file saved:  ./dataset/img/서울N타워_B